In [2]:
!pip install wget | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install "ibm-watson-machine-learning>=1.0.310" | tail -n 1

In [3]:
url = "https://us-south.ml.cloud.ibm.com"
apikey='VZFvE0ufR4ePXpSobgdtuBLMCQmBjxVuhnUD7aYC4osl'

In [4]:
credentials = {
    "url": url,
    "apikey": apikey
}

In [5]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")


project_id

'a100f552-8d74-4455-a7cb-bee0dba6ffd2'

In [6]:
import  types
import pandas as pd
from botocore.client import Config
import ibm_boto3

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='VZFvE0ufR4ePXpSobgdtuBLMCQmBjxVuhnUD7aYC4osl',  # Your secret API key
    ibm_auth_endpoint='https://iam.cloud.ibm.com/oidc/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.us-south.cloud-object-storage.appdomain.cloud'
)

bucket = 'sentiment-analysislegal'
object_key = 'legal sentiment analyzer.csv'

body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
if not hasattr(body, "iter"):
    body.iter = types.MethodType( iter, body)
data = pd.read_csv(body)
data.head(10)

,ID,Phrase,Sentiment
0,1,The plaintiff's claims are dismissed.,-1
1,2,The contract is deemed valid and enforceable.,1
2,3,The appeal is denied due to lack of merit.,-1
3,4,The legal team submitted additional evidence.,0
4,5,The appeal is under consideration.,0
5,6,The settlement agreement is fair and reasonable.,1
6,7,The appeal is under consideration.,0
7,8,The settlement agreement is fair and reasonable.,1
8,9,The court reviewed the documentation.,0
9,10,The contract is deemed valid and enforceable.,1


In [7]:
lable_map={
    -1:'negative',
     0:'netural',
     1: 'positive'
}
data.value_counts(['Sentiment'])


Sentiment
 1           184
-1           167
 0           149
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd

data_train, data_test, y_train, y_test = train_test_split(
    data['Phrase'],
    data['Sentiment'],
    test_size=0.3,
    random_state=33,
    stratify=data['Sentiment']
)

# Convert to DataFrames
data_train = pd.DataFrame({'Phrase': data_train, 'Sentiment': y_train})
data_test = pd.DataFrame({'Phrase': data_test, 'Sentiment': y_test})


In [9]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q', 'CODELLAMA_34B_INSTRUCT_HF', 'GRANITE_20B_MULTILINGUAL']


In [10]:
model_id = ModelTypes.FLAN_T5_XXL


In [11]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.RANDOM_SEED: 33,
    GenParams.REPETITION_PENALTY: 1,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 1
}

In [12]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

model.get_details()


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:273: LifecycleWarning: Model 'google/flan-t5-xxl' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


{'model_id': 'google/flan-t5-xxl',
 'label': 'flan-t5-xxl-11b',
 'provider': 'Google',
 'source': 'Hugging Face',
 'functions': [{'id': 'text_generation'}],
 'short_description': 'flan-t5-xxl is an 11 billion parameter model based on the Flan-T5 family.',
 'long_description': 'flan-t5-xxl (11B) is an 11 billion parameter model based on the Flan-T5 family. It is a pretrained T5 - an encoder-decoder model pre-trained on a mixture of supervised / unsupervised tasks converted into a text-to-text format, and fine-tuned on the Fine-tuned Language Net (FLAN) with instructions for better zero-shot and few-shot performance.',
 'terms_url': 'https://huggingface.co/google/flan-t5-xxl/blob/main/README.md',
 'input_tier': 'class_2',
 'output_tier': 'class_2',
 'number_params': '11b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'

In [13]:
instruction = """Determine the sentiment of the sentence.
Use either 'positive', 'negative', 'neutral'. Use the provided examples as a template."""

In [14]:
zero_shot_inputs = [{"input": text} for text in data_test['Phrase']]
for i in range(2):
    print(f"The sentence example {i+1} is:\n\t {zero_shot_inputs[i]['input']}\n")

The sentence example 1 is:
	 The legal team submitted additional evidence.

The sentence example 2 is:
	 The appeal is denied due to lack of merit.



In [15]:
data_train_and_labels = data_train.copy()
data_train_and_labels['Sentiment'] = y_train


In [16]:

few_shot_example = []
few_shot_examples = []

for phrase, sentiment in data_train_and_labels \
    .groupby('Sentiment') \
    .apply(lambda x: x.sample(2)).values:
    
    few_shot_example.append(f"\tsentence:\t{phrase}\n\tsentiment: {sentiment}\n")

few_shot_examples = '\n'.join(few_shot_example)

In [17]:
few_shot_inputs_ = [{"input": text} for text in data_test['Phrase'].values]

for i in range(2):
    print(f"The sentence example {i+1} is:\n {few_shot_inputs_[i]['input']}\n")
    print(f"\tSentiment: {y_test.iloc[i]}\n")


The sentence example 1 is:
 The legal team submitted additional evidence.

	Sentiment: 0

The sentence example 2 is:
 The appeal is denied due to lack of merit.

	Sentiment: -1



In [18]:

results = []
for inp in few_shot_inputs_[:2]:
    results.append(
        model.generate(" ".join([instruction + few_shot_examples[0], inp['input']]))["results"][0]
    )

In [19]:
import json
print(json.dumps(results, indent=2))


[
  {
    "generated_text": "neutral",
    "generated_token_count": 1,
    "input_token_count": 42,
    "stop_reason": "max_tokens"
  },
  {
    "generated_text": "negative",
    "generated_token_count": 1,
    "input_token_count": 45,
    "stop_reason": "max_tokens"
  }
]


In [25]:
from sklearn.metrics import classification_report

# Assume `results` is your list of model outputs (len(results)=N)
N = len(results)

# 1. Re-build the true labels for just those N samples
y_true_sub = [lable_map[lable] for lable in y_test.values[:N]]

# 2. Extract your predictions
y_pred = [r['generated_text'] for r in results]

# 3. Sanity‐check lengths
print(f"Length of y_true_sub: {len(y_true_sub)}")
print(f"Length of y_pred:      {len(y_pred)}\n")

# 4. Generate the report
print(classification_report(y_true_sub, y_pred, zero_division=0))


Length of y_true_sub: 2
Length of y_pred:      2

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         1
     netural       0.00      0.00      0.00         1
     neutral       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2



In [20]:
y_true = [lable_map[lable] for lable in y_test.values[:2]]
y_true

['netural', 'negative']

In [21]:
y_pred = [result['generated_text'] for result in results]
y_pred

['neutral', 'negative']